# Predicting Bank Customer Attrition
By Lance Belen

<h2><u>Data Retrieval and Preparation</u></h2>

In [1]:
import pandas as pd
df = pd.read_csv("BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [2]:
df.shape

(10127, 23)

In [3]:
df.dtypes

CLIENTNUM                                                                                                                               int64
Attrition_Flag                                                                                                                         object
Customer_Age                                                                                                                            int64
Gender                                                                                                                                 object
Dependent_count                                                                                                                         int64
Education_Level                                                                                                                        object
Marital_Status                                                                                                                         object
Income

<h3><i>Data Pre-Processing</i></h3>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df_cleaned = df.copy()
df_cleaned = df_cleaned.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)
df_cleaned.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


<h4><u>Remove outliers</u></h4>

In [10]:
for col in df_cleaned.columns:
    if df[col].dtype == 'int':
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        threshold = 1.5
        outliers = df_cleaned[(df_cleaned[col] < Q1 - threshold * IQR) | (df_cleaned[col] > Q3 + threshold * IQR)]
        df_cleaned = df_cleaned.drop(outliers.index)
        print(f'Dropped {len(outliers)} outliers in {col}.')

Dropped 0 outliers in CLIENTNUM.
Dropped 2 outliers in Customer_Age.
Dropped 0 outliers in Dependent_count.
Dropped 385 outliers in Months_on_book.
Dropped 0 outliers in Total_Relationship_Count.
Dropped 308 outliers in Months_Inactive_12_mon.
Dropped 584 outliers in Contacts_Count_12_mon.
Dropped 0 outliers in Total_Revolving_Bal.
Dropped 808 outliers in Total_Trans_Amt.
Dropped 0 outliers in Total_Trans_Ct.


<h4><u>Check for missing values</u></h4>

In [21]:
if (len(df_cleaned[df_cleaned[col].isnull() == True]) > 0):
    print(f'Missing value/s found!')
else:
    print('No missing value/s!')

No missing value/s!


<h2><u>Data Exploration</u></h2>

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
df_cleaned.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
5,713061558,Existing Customer,44,M,2,Graduate,Married,$40K - $60K,Blue,36,...,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,810347208,Existing Customer,51,M,4,Unknown,Married,$120K +,Gold,46,...,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066


In [24]:
df_cleaned.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,8.040000e+03,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000,8040.000000
mean,7.386608e+08,46.338806,2.408209,35.907214,3.932214,2.268035,2.512562,8162.550896,1147.994900,7014.555995,0.753408,3537.054851,61.398383,0.708008,0.284044
std,3.649297e+07,7.445478,1.271965,6.991000,1.512483,0.858823,0.927442,8803.717930,815.958232,8815.469105,0.220990,1676.852012,20.058525,0.243195,0.280200
min,7.080833e+08,26.000000,0.000000,18.000000,1.000000,1.000000,1.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,7.130428e+08,41.000000,2.000000,32.000000,3.000000,2.000000,2.000000,2447.000000,159.000000,1179.000000,0.621000,2077.000000,44.000000,0.571000,0.008750
50%,7.178284e+08,46.000000,2.000000,36.000000,4.000000,2.000000,3.000000,4198.500000,1257.000000,3123.000000,0.728000,3736.500000,65.000000,0.694000,0.187000
75%,7.725820e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,10080.500000,1769.000000,8990.500000,0.855000,4542.000000,78.000000,0.821000,0.522000
max,8.283431e+08,68.000000,5.000000,53.000000,6.000000,4.000000,4.000000,34516.000000,2517.000000,34516.000000,2.675000,8620.000000,113.000000,3.714000,0.999000


In [26]:
df_cleaned['Attrition_Flag'].value_counts()

Attrition_Flag
Existing Customer    6703
Attrited Customer    1337
Name: count, dtype: int64